# Exploratory Data Analysis
Familiarize with the dataset, we'll try to target question 3. Recall the
question asks "do periods with low meat production track with overall
unemployment numbers?"

## Environment Setup
Load modules, data.

In [2]:
# modules
## external
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## stdlib
from pathlib import Path
from typing import Any, Callable

## utils

In [3]:
# load in datasets
home_dir = Path().cwd().parent                                                  # github repo home dir

## financial data
economic_df = pd.read_csv(
    home_dir / "datasets" / "acs_5yr_est_selected_economic_characteristics_2010-2022.csv",
    engine="c", low_memory=False
)
commodities_df = pd.read_csv(
    home_dir / "datasets" / "all_commodities.csv",
    engine="c", low_memory=False
)
stocks_df = pd.read_csv(
    home_dir / "datasets" / "all_stock_and_etfs.csv",
    engine="c", low_memory=False
)
stock_descr_df = pd.read_csv(
    home_dir / "datasets" / "stock_descriptions.csv",
    engine="c", low_memory=False
)

## meat data
meat_storage_df = pd.read_csv(
    home_dir / "datasets" / "Meat_Stats_Cold_Storage.csv", engine="c",
    low_memory=False
)
meat_production_df = pd.read_csv(
    home_dir / "datasets" / "Meat_Stats_Meat_Production.csv", engine="c",
    low_memory=False
)
meat_slaughter_weights_df = pd.read_csv(
    home_dir / "datasets" / "Meat_Stats_Slaughter_Counts.csv", engine="c",
    low_memory=False
)
meat_slaughter_counts_df = pd.read_csv(
    home_dir / "datasets" / "Meat_Stats_Slaughter_Weights.csv", engine="c",
    low_memory=False
)
nutrition_df = pd.read_csv(
    home_dir / "datasets" / "Nutrition_Physical_Activity_and_Obesity_Data.csv", 
    engine="c", low_memory=False
)

# generate lookup for easy replication downstream
dfs = {
    "economic": economic_df,
    "commodities": commodities_df,
    "stocks": stocks_df,
    "stock_descr": stock_descr_df,
    "meat_storage": meat_storage_df,
    "meat_production": meat_production_df,
    "meat_slaughter_weights": meat_slaughter_weights_df,
    "meat_slaughter_counts": meat_slaughter_counts_df,
    "nutrition": nutrition_df
}

## Exploratory Analysis
Distributions, plots, etc.

In [4]:
# auxiliary fn for easy application
def apply_dfs(dfs: dict[str, pd.DataFrame], fn: Callable) -> dict[str, Any]:
    """Applies fn to all dfs in the lookup.

    Args:
        dfs (dict[str, pd.DataFrame]): lookup
        fn (Callable): function to apply, df is passed as the first parameter

    Returns:
        dict[str, Any]: same keys to the output of the function
    """
    
    return {df_type: fn(df) for df_type, df in dfs.items()}

def pprint_dfs(dfs: dict[str, pd.DataFrame]) -> None:
    """Pretty printing for the dfs lookup.

    Args:
        dfs (dict[str, pd.DataFrame]): lookup
    """
    
    for df_type, df in dfs.items():
        print(f"< ::::::: {df_type.replace('_', ' ').capitalize()} ::::::: >")
        print(df)
        print("\n\n")

In [5]:
# inspection
## inspect rows
pprint_dfs(
    apply_dfs(dfs, pd.DataFrame.head)
)

## inspect distributions
pprint_dfs(
    apply_dfs(dfs, pd.DataFrame.describe)
)

## inspect metadata
apply_dfs(dfs, pd.DataFrame.info)

< ::::::: Economic ::::::: >
               Label (Grouping)           Category    State   Estimate  \
0             EMPLOYMENT STATUS             Header  Alabama        NaN   
1  Population 16 years and over  EMPLOYMENT STATUS  Alabama  3,714,504   
2                In labor force  EMPLOYMENT STATUS  Alabama  2,246,848   
3          Civilian labor force  EMPLOYMENT STATUS  Alabama  2,230,879   
4                      Employed  EMPLOYMENT STATUS  Alabama  2,036,867   

  Margin of Error    Percent Percent Margin of Error  Year  
0             NaN        NaN                     NaN  2010  
1          ±1,588  3,714,504                     (X)  2010  
2          ±5,576      60.5%                    ±0.1  2010  
3          ±5,531      60.1%                    ±0.1  2010  
4          ±5,904      54.8%                    ±0.2  2010  



< ::::::: Commodities ::::::: >
    Date-Time       Value             Unit Commodity
0  2024-01-01  203.879130  Cents per Pound    Coffee
1  2023-12-01  210.

{'economic': None,
 'commodities': None,
 'stocks': None,
 'stock_descr': None,
 'meat_storage': None,
 'meat_production': None,
 'meat_slaughter_weights': None,
 'meat_slaughter_counts': None,
 'nutrition': None}

## Cleaning
Removing NA, reformatting, etc.

The data about economics trends is particularly useful for finding the
unemployment in the states with highest meat production, etc. Data about
commodities can tell us how the price of meat corresponds to meat production,
although this veers more into question 1.

Data about meat production is of course essential as well. Some supplementary
insights can be found in slaughter weights and counts since that directly
relates to the rate of meat production, as well as meat storage since more meat
that stored means less meat is needed to be produced. Some of this info may be
redundant, but it's good to not remove too much data at once.

In [10]:
# only keep useful datasets
useful_dfs = [
    "economic", "commodities", "meat_storage", "meat_production",
    "meat_slaughter_weights", "meat_slaughter_counts"
]
dfs = {k: dfs[k] for k in useful_dfs}

In [ ]:
# remove na values
dfs.isna()